# Edad vs TECHx5

[Treemap(px)](https://plotly.com/python/treemaps/)

## 1. importar las librerías + csv con los datos de la encuesta. 

In [1]:
# importar librerias

import pandas as pd
import plotly.express as px  
import plotly.graph_objects as go

import webbrowser
from threading import Timer
import numpy as np
import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


#crear un dataframe con toda la informacion de la encuesta
#anyadido: path relativo
df21 = pd.read_csv ('../../data/survey_results_public2021.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta
df21 #mostrar df (

,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
ResponseId,,,,,,,,,,,,,,,,,,,,,
1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,NaN,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,10,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83435,I am a developer by profession,Employed full-time,United States of America,Texas,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",6,5,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,I have a concentration and/or memory disorder ...,Appropriate in length,Easy,160500.0
83436,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Benin,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",4,2,...,18-24 years old,Man,No,Straight / Heterosexual,Black or of African descent,None of the above,None of the above,Appropriate in length,Easy,3960.0
83437,I am a developer by profession,Employed full-time,United States of America,New Jersey,NaN,"Secondary school (e.g. American high school, G...",11 - 17 years,School,10,4,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,None of the above,Appropriate in length,Neither easy nor difficult,90000.0


## 2. Preprocesar datos.

Tratar las columnas/conjunto de datos para comenzar a crear los gráficos.

In [2]:
def ageTech (df3,col,col1):

    # copiamos elementos que no sean NaN
    df=df3[[col1, col]].dropna().copy() #Age1stCode, LearnCode, EdLevel
    
    if (col1 == "Age"): df.drop(df[df.Age == 'Prefer not to say'].index) #eliminar info irrelevante

     #df4['EdLevel'] = df4['EdLevel'].str.replace(r"\(.*?\)", "", regex=True) 

    df2 = (df.explode(df.columns.tolist())
      .apply(lambda col: col.str.split(';')) 
      .explode(col1)
      .explode(col))

    if (col1 == "EdLevel"): df2['EdLevel'].str.replace(r"\(.*?\)", "", regex=True)
    
    
    return (pd.crosstab(df2[col1], df2[col])
       .melt(value_name='count', ignore_index=False)
       .reset_index())




## 3. layout + callback 


In [3]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}]
                )
app.layout = html.Div([
    
    dbc.Row([ #dropdown
                    dbc.Col([
                        dcc.Dropdown(id="opt5",  multi=False, value='Age',
                                 options=[ 
                                     {"label": "EdLevel", "value": "EdLevel"},
                                     {"label": "LearnCode", "value": "LearnCode"},
                                     {"label": "Age", "value": "Age"}],
                                     style={'width': '45vh'},
                                    ),
                    ]) 
                ]), #dropdown
    
    dbc.Row([
            dbc.Tabs([
                dbc.Tab(label='Databases', tab_id='db1',labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                dbc.Tab(label='Languages', tab_id='lang1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Platform', tab_id='plat1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='webframe', tab_id='web1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Miscelaneous', tab_id='misc1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Tools', tab_id='tools1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Collab', tab_id='colab1', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                ],
                id="sankey",
                active_tab="db1",
            ),
        dbc.CardBody(html.P(id="tablas-content", className="card-text")),
    ]
)
])



#tabs

@app.callback(
    Output("tablas-content", "children"), 
    [(Input("sankey", "active_tab")),
    Input("opt5", "value")])
    
def tab_content(active_tab, value):
    
#     df = pd.DataFrame()

    nameColumns= ['DatabaseHaveWorkedWith','LanguageHaveWorkedWith', 'PlatformHaveWorkedWith',  
                  'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 
                  'NEWCollabToolsHaveWorkedWith']

    name = ["Databases", "Languages", "Platforms", "Webframes", "Miscelaneous Tech", "Tools", "Collab tools"]
    orden = 0
    
    nameColumns2 = ["Age", "EdLevel", "LearnCode"]
    orden2 = 0
    name2 = ["Age", "Ed Level", "Learn Code"]

    if active_tab is not None: #para evitar un error hay que añadir un caso donde active_tab sste vacio
        if active_tab =="db1":
            orden = 0
            
        elif active_tab == "lang1":
            orden = 1
            
        elif active_tab == "plat1":
            orden = 2
            
        elif active_tab == "web1":
            orden = 3
            
        elif active_tab == "misc1":
            orden = 4
            
        elif active_tab == "tools1":
            orden = 5
            
        elif active_tab == "colab1":
            orden = 6
        
        if value == "EdLevel": orden2= 1
        elif value == "Age": orden2 = 0
        elif value == "LearnCode": orden2 = 2
            
            
        
        df = ageTech(df21,nameColumns[orden],nameColumns2[orden2])
        fig = px.treemap(df, path=[nameColumns2[orden2], nameColumns[orden]],values='count',)
        fig.update_layout(title_text="Comparison between "+name2[orden2]+" and top 5 of most used "+name[orden], font_size=12)
        
        return dcc.Graph(figure=fig, style={'height': '90vh'}) 
        
        
    return html.P("This shouldn't ever be displayed...")

## 4. run

In [4]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
